In [1]:
import pandas as pd # type: ignore
import numpy as np # type: ignore
import matplotlib.cm as cm # type: ignore
import matplotlib.pyplot as plt # type: ignore
import matplotlib.image as mpimg # type: ignore
import xml.etree.ElementTree as ET # type: ignore
from collections import defaultdict # type: ignore
import matplotlib.patheffects as path_effects # type: ignore
from matplotlib.ticker import FixedFormatter, FixedLocator # type: ignore

In [2]:
def load_node_table(file_path) :
    node_map = {}
    with open(file_path, 'r') as file :
        for line in file :
            parts = line.split(':')
            node_index = int(parts[0].strip())
            node_value = int(parts[1].strip())
            node_map[node_index] = node_value
    return node_map

In [3]:
def extract_data_form_xml(file_path, map_width = 1300, map_height = 1600, grid_size = 10, time_limit=100.0, target_vehicle_ids=None) :
    tree = ET.parse(file_path)
    root = tree.getroot()
    extracted_data = []
    
    for timestep in root.findall('timestep') :
        time = float(timestep.get('time'))
        if time > time_limit:
            break
        for vehicle in timestep.findall('vehicle') :
            vehicle_id = int(vehicle.get('id'))
            
            if target_vehicle_ids is not None and vehicle_id not in target_vehicle_ids :
                continue
            x = float(vehicle.get('x'))
            y = float(vehicle.get('y'))
            speed = float(vehicle.get('speed'))
            
            grid_x = int(x / (map_width / grid_size))
            grid_y = int(y / (map_height / grid_size))
            
            extracted_data.append((time, vehicle_id, x, y, speed, grid_x, grid_y))
    columns = ['time', 'vehicle_id', 'x', 'y', 'speed', 'grid_x', 'grid_y']
    df = pd.DataFrame(extracted_data, columns=columns)
    return df
            

In [7]:
def main() :
    node_table = load_node_table('resource/node_table_low.txt')
    target_node_ids = [i for i in range(0, 231)]
    target_vehicle_ids = [node_table[node_id] for node_id in target_node_ids if node_id in node_table]
    node_to_vehicle_map = {node_id : node_table[node_id] for node_id in target_node_ids if node_id in node_table}
    
    data = extract_data_form_xml ('resource/map0.xml', time_limit=200.0, target_vehicle_ids=target_vehicle_ids)
    print(data)

In [8]:
if __name__ == '__main__' :
    main()

         time  vehicle_id        x        y  speed  grid_x  grid_y
0         0.0           0  1327.16   487.92   0.00      10       3
1         0.1           0  1327.15   487.94   0.25      10       3
2         0.2           0  1327.13   487.98   0.49      10       3
3         0.3           0  1327.11   488.05   0.75      10       3
4         0.4           0  1327.07   488.15   0.98      10       3
...       ...         ...      ...      ...    ...     ...     ...
181402  199.9          52   772.54  1250.81   0.00       5       7
181403  199.9          67   785.01  1529.41  10.57       6       9
181404  199.9          87   754.72  1282.02   0.00       5       8
181405  199.9          97  1123.41  1524.73  12.51       8       9
181406  199.9          98  1296.96  1529.08   4.42       9       9

[181407 rows x 7 columns]
